In [ ]:
# Install a separate conda installation via Miniconda
WORKING_DIR=/home/ec2-user/SageMaker
mkdir -p "$WORKING_DIR"
wget https://repo.anaconda.com/miniconda/Miniconda3-py310_23.5.2-0-Linux-x86_64.sh -O "$WORKING_DIR/miniconda.sh"
bash "$WORKING_DIR/miniconda.sh" -b -u -p "$WORKING_DIR/miniconda" 
rm -rf "$WORKING_DIR/miniconda.sh"

In [ ]:
# Create a custom conda environment
source "$WORKING_DIR/miniconda/bin/activate"
KERNEL_NAME="diffusers"
PYTHON="3.10"
conda create --yes --name "$KERNEL_NAME" python="$PYTHON"
conda activate "$KERNEL_NAME"
pip install --quiet ipykernel

In [ ]:
cd SageMaker
git clone https://github.com/huggingface/diffusers
cd diffusers
pip install -e .
cd /home/ec2-user/SageMaker

In [ ]:
pip install torch==2.0.1
pip install accelerate==0.30.0
pip install torchvision
pip install transformers==4.44.0
pip install ftfy
pip install tensorboard
pip install Jinja2
pip install peft==0.7.0
pip install xformers==0.0.20
pip install bitsandbytes
pip install scipy
pip install numpy==1.26.4

In [ ]:
accelerate config default

In [ ]:
cd /home/ubuntu
mkdir dog-example

# Download dog-example dataset from Hugging Face
cat > download-dog-example.py << EOF
from huggingface_hub import snapshot_download

local_dir = "./dog-example"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)
EOF

python download-dog-example.py

In [ ]:
cd /home/ec2-user/SageMaker
wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

In [ ]:
accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --instance_data_dir="dog_example" \
  --output_dir="lora-trained-xl" \
  --mixed_precision="fp16" \
  --instance_prompt="a photo of sks dog" \
  --resolution=1024 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=2 \
  --gradient_checkpointing \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --use_8bit_adam \
  --max_train_steps=500 \
  --checkpointing_steps=717 \
  --seed="0"

In [ ]:
# sdxl-lora-inference-base.py
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")
pipe.load_lora_weights(
    "lora-trained-xl/", 
    weight_name="pytorch_lora_weights.safetensors"
)

prompt = "a photo of sks dog in a bucket"

for seed in range(4):
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt=prompt, generator=generator, num_inference_steps=25)
    image = image.images[0]
    image.save(f"sdxl-base-{seed}.png")

In [ ]:
# sdxl-lora-inference-base-refiner.py
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

# Load the base pipeline and load the LoRA parameters into it.
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16
)
pipe.to("cuda")
pipe.load_lora_weights(
    "lora-trained-xl/", 
    weight_name="pytorch_lora_weights.safetensors"
)

# Load the refiner.
refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype=torch.float16
)
refiner.to("cuda")

prompt = "a photo of sks dog in a bucket"

for seed in range(4):
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt=prompt, generator=generator, num_inference_steps=25)
    image = image.images[0]
    image = refiner(prompt=prompt, generator=generator, image=image)
    image = image.images[0]
    image.save(f"sdxl-base-refiner-{seed}.png")

In [ ]:
# After you stop and start the instance again
WORKING_DIR=/home/ec2-user/SageMaker
mkdir -p "$WORKING_DIR"
source "$WORKING_DIR/miniconda/bin/activate"
KERNEL_NAME="diffusers"
PYTHON="3.10"
conda activate "$KERNEL_NAME"
pip install --quiet ipykernel

In [ ]:
python3 sdxl-lora-inference-base.py